In [8]:
from database.engine import session_maker
from database.models import Municipalities, MunicSubscriptions
from sqlalchemy import select

In [6]:
user_id = 964635576

In [14]:
async with session_maker() as session_:
    query = select(Municipalities.municipality_name).join(MunicSubscriptions, Municipalities.map_id == MunicSubscriptions.map_id).where(MunicSubscriptions.user_id == user_id)
    response = await session_.execute(query)
    response = response.all()
    response = [i[0] for i in response]
    print(response)
    

['город Ачинск', 'город Боготол', 'город Дивногорск']
